# Опис проєкту

Буде завантажено реальний набір даних, виконано попередню обробку, очистку даних, заповнено відсутні значення, виконано нормалізацію та, за потреби, створено нові ознаки.

Для виконання буде використано NumPy та Pandas

# Імпорт бібліотек

In [1]:
import numpy as np
import pandas as pd
import io

* **NumPy** для роботи з матрицями та обробкою даних
* **Pandas** для завантаження набору даних

# Завантаження набору даних

Для прикладу буде використано популярний набір даних "Diabetes" з UCI Machine Learning Repository. Цей набір містить інформацію про медичні виміри для діагностики діабету

In [5]:
from google.colab import files

uploaded = files.upload()

Saving diabetes_data_all_patient.csv to diabetes_data_all_patient.csv


In [10]:
url = "https://drive.google.com/file/d/1F1nz8jghTox9ymfphZ7mUNsxf6Q2S1In/view?usp=sharing"

# завантаження даних
data = pd.read_csv(io.BytesIO(uploaded['diabetes_data_all_patient.csv']), header=None)

# попередній перегляд даних
print(data.head())

     0           1           2      3     4      5
0  NaN  patient_id        date   time  code  value
1  0.0          01  04-21-1991   9:09    58    100
2  1.0          01  04-21-1991   9:09    33      9
3  2.0          01  04-21-1991   9:09    34     13
4  3.0          01  04-21-1991  17:08    62    119


# Попередня обробка та очищення даних

1. Видалення або заповнення відсутніх значень. Перевіримо, чи є відсутні значення, і, якщо є, вирішимо, як з ними працювати

In [12]:
# перевірка на наявнясть відсутніх значень
print(data.isnull().sum())

# приклад заповнення відсутніх значень середнім значенням
data.fillna(data.mean(), inplace=True)

0     1
1     0
2    33
3     0
4     0
5    33
dtype: int64


TypeError: can only concatenate str (not "int") to str